In [81]:
import numpy as np
import GPyOpt
from GPyOpt.objective_examples.experiments2d import mistery, dropwave
import GPy as GPy
from multi_objective import MultiObjective
from multi_outputGP import multi_outputGP
import matplotlib.pyplot as plt
import scipy
from continuous_KG import KG
from bayesian_optimisation import BO
import pandas as pd
import os
from GPyOpt.experiment_design import initial_design

In [82]:
noise = 1
mistery_f =mistery(sd=np.sqrt(noise))

# --- Attributes
#repeat same objective function to solve a 1 objective problem
f = MultiObjective([mistery_f.f])
c = MultiObjective([mistery_f.c])


# --- Attributes
#repeat same objective function to solve a 1 objective problem

#c2 = MultiObjective([test_c2])
# --- Space
#define space of variables
space =  GPyOpt.Design_space(space =[{'name': 'var_1', 'type': 'continuous', 'domain': (0,5)},{'name': 'var_2', 'type': 'continuous', 'domain': (0,5)}])#GPyOpt.Design_space(space =[{'name': 'var_1', 'type': 'continuous', 'domain': (0,100)}])#
n_f = 1
n_c = 1
model_f = multi_outputGP(output_dim = n_f,   noise_var=[noise]*n_f, exact_feval=[True]*n_f)
model_c = multi_outputGP(output_dim = n_c,  noise_var=[1e-6]*n_c, exact_feval=[True]*n_c)


In [83]:
def _update_model(model_f, model_c):
    """
    Updates the model (when more than one observation is available) and saves the parameters (if available).
    """

    ### --- input that goes into the model (is unziped in case there are categorical variables)
    X = np.random.random((10,2))*5
    
    Y, cost_values = f.evaluate(X)
    C, cost_values = c.evaluate(X)
            
    X_inmodel = space.unzip_inputs(X)
    Y_inmodel = list(Y)
    C_inmodel = list(C)

    model_f.updateModel(X_inmodel, Y_inmodel)
    model_c.updateModel(X_inmodel, C_inmodel)

In [84]:
_update_model(model_f, model_c)

reconstraining parameters GP_regression.rbf
reconstraining parameters GP_regression.Gaussian_noise.variance
reconstraining parameters GP_regression.rbf
reconstraining parameters GP_regression.Gaussian_noise.variance


create model
self.model 
Name : GP regression
Objective : 67.80969959569009
Number of Parameters : 3
Number of Optimization Parameters : 2
Updates : True
Parameters:
  GP_regression.           |               value  |  constraints  |     priors   
  rbf.variance             |  36.823426783817695  |      +ve      |   Ga(1, 0.5) 
  rbf.lengthscale          |  2.9681869838585344  |      +ve      |   Ga(1, 0.5) 
  Gaussian_noise.variance  |                 1.0  |   +ve fixed   |  Ga(0.2, 0.2)
create model
self.model 
Name : GP regression
Objective : 12.866326935325255
Number of Parameters : 3
Number of Optimization Parameters : 2
Updates : True
Parameters:
  GP_regression.           |               value  |  constraints  |     priors   
  rbf.variance             |  0.6856940813733223  |      +ve      |   Ga(1, 0.5) 
  rbf.lengthscale          |  1.2721781122399025  |      +ve      |   Ga(1, 0.5) 
  Gaussian_noise.variance  |               1e-06  |   +ve fixed   |  Ga(0.2, 0.2)


In [85]:
from aux_modules.gradient_modules import gradients

x =np.array([[3,1]]) 
Z_obj = -1.96
Z_const = 1.96
X_inner =np.array([[2,3]])
aux_obj = [0.1]
aux_c = [0.1]
x_opt=np.array([[4,4]])

model_f.partial_precomputation_for_covariance(x)
model_f.partial_precomputation_for_covariance_gradient(x)

model_c.partial_precomputation_for_covariance(x)
model_c.partial_precomputation_for_covariance_gradient(x)

grad_obj = gradients(x_new=x, model= model_f, xopt =x_opt, Z = Z_obj, aux=aux_obj, X_inner=X_inner, precompute_grad =True)

mu_xnew = grad_obj.compute_value_mu_xnew(x=X_inner)
grad_mu_xnew = grad_obj.compute_gradient_mu_xnew(x=X_inner)

grad_c = gradients(x_new=x, model=model_c, xopt =x_opt, Z=Z_const, aux=aux_c, X_inner=X_inner,
                   precompute_grad=True)
Fz, grad_Fz = grad_c.compute_probability_feasibility_multi_gp(x=X_inner, l=0,
                                                              gradient_flag=True)

func_val = np.array(mu_xnew * Fz) #- self.control_variate

func_grad_val = np.array(mu_xnew).reshape(-1) * grad_Fz.reshape(-1) + Fz.reshape(
    -1) * grad_mu_xnew.reshape(
    -1) 

In [86]:
f = grad_obj.trial_compute_b_xopt
x = x.reshape(1,-1)
f_val = np.array(f(x)).reshape(-1)
numerical_grad = []
analytical_grad = []
func_val = []
dim = x.shape[1]
delta= 1e-4
delta_matrix = np.identity(dim)
func_val = []
f_delta=[]
for i in range(dim):
    one_side = np.array(f(x + delta_matrix[i]*delta)).reshape(-1)
    two_side = np.array(f(x - delta_matrix[i]*delta)).reshape(-1)
    f_delta.append(one_side - two_side)

func_val.append(f_val)
f_delta = np.array(f_delta).reshape(-1)
numerical_grad.append(np.array(f_delta/(2*delta)).reshape(-1))

In [87]:
numerical_grad

[array([-0.01386109, -0.00317024])]

In [95]:
a = np.array([6.42336354])
b = -0.025562319816145457
np.matmul(np.sqrt(a),np.array(b))

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [96]:
np.square(b).reshape(-1)

array([0.00065343])